# Prospect‑Theory Backtesting – Analysis Notebook
This Jupyter notebook reproduces and extends the analysis from `results_pt_avg.py`.  

It:
1. Generates rolling Bayesian Model Averaging (BMA) forecasts and historical‐mean forecasts.
2. Runs Prospect‑Theory (PT) optimisations, a MVP benchmark, a Max‑Sharpe benchmark, and a naive 1/N portfolio.
3. Calculates **annualised Sharpe ratios** (monthly data ×√12).
4. Computes **average Certainty Equivalents (CE)** across the λ/γ grid for PT methods.
5. Builds and compares **cumulative net‑return paths** for each method.
6. Presents comparison tables:
   * Strategy‑level metrics within each PT method
   * Average cumulative‑return comparison across methods
   * Same summary tables shown in the original script

---

In [10]:
# ---- Locate and add the 'functions' directory automatically -------------
import sys, pathlib

def add_functions_dir():
    here = pathlib.Path.cwd()          # start from current working dir
    for p in [here] + list(here.parents):
        candidate = p / "functions"
        if candidate.is_dir():
            sys.path.insert(0, str(candidate))
            print("✅  Added to sys.path:", candidate)
            return
    print("⚠️  'functions' folder not found – check your project structure")

add_functions_dir()
import prospect_optimizer as po
import evaluation as ev

✅  Added to sys.path: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\functions


In [11]:
# --- User settings -------------------------------------------------
strategies      = ["conservative", "aggressive"]
lambda_values   = [1.99, 2.5]
gamma_values    = [0.12, 0.20]

start_date      = "1977-06-01"
end_date        = "2016-12-01"
date_tag        = f"{start_date}_{end_date}"

parent_dir      = os.getcwd()
cache_dir       = "./bma_cache"

In [19]:
# --- Helper functions ---------------------------------------------
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
def annualised_sharpe(ret_series, rf=0.0):
    """Annualises the Sharpe ratio for monthly data (ex‑ante rf)."""
    excess = ret_series - rf
    return excess.mean() / excess.std(ddof=1) * np.sqrt(12)

def mean_cumulative(results_dict):
    """Equal‑weight average cumulative net‑return path over all strategies."""
    cum_paths = [
        df["Compounded Returns"].rename(key)
        for key, df in results_dict.items()
    ]
    return pd.concat(cum_paths, axis=1).mean(axis=1)

In [13]:
# --- Run back‑tests ----------------------------------------------
print("⏳ Rolling BMA forecasts …")
bma_returns = po.rolling_bma_returns(
    parent_dir, n_predictors_to_use=2,
    start_date=start_date, end_date=end_date)

historical_returns = po.load_historical_returns(
    parent_dir, start_date, end_date)

reference_series = po.load_risk_free_rate_from_factors(
    parent_dir, start_date, end_date)
reference_series = po.slice_reference(reference_series, start_date, end_date)

print("⏳ Back‑testing PT by BMA …")
results_bma = po.resultgenerator_bma(
    lambda_values, gamma_values, bma_returns,
    historical_returns, strategies, date_tag, cache_dir)

print("⏳ Back‑testing PT by Historical mean …")
results_hist = po.resultgenerator_historical_mean(
    lambda_values, gamma_values, historical_returns,
    strategies, date_tag)

print("⏳ Back‑testing MVP (benchmark) …")
results_mvp = po.resultgenerator_mvp(
    lambda_values, gamma_values, historical_returns,
    strategies, date_tag)

print("⏳ Back‑testing Max‑Sharpe on BMA …")
results_bma_ms = po.resultgenerator_bma_maxsharpe(
    lambda_values, gamma_values, bma_returns,
    historical_returns, strategies, date_tag)

print("⏳ Building naive equal‑weight portfolio …")
naive = po.naive_equal_weight_portfolio(
    historical_returns, start_date, end_date)
naive_key = f"conservative_{lambda_values[0]}_{gamma_values[0]}"
results_naive = {naive_key: naive}

results_by_method = {
    "PT BMA":            results_bma,
    "PT HistMean":       results_hist,
    "MVP benchmark":     results_mvp,
    "MaxSharpe BMA":     results_bma_ms,
    "Naive 1/N":         results_naive,
}

⏳ Rolling BMA forecasts …
Index(['Date', 'Mkt-RF', 'SMB', 'CMA', 'PEAD', 'QMJ', 'MGMT', 'PERF', 'LIQ',
       'IFCR'],
      dtype='object')
Index(['Date', 'dp', 'dy', 'ep', 'de', 'svar', 'ntis', 'tbl', 'lty', 'dfy'], dtype='object')
Processing 1987-06
Loading cached BMA initialization from: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\bma_cache2.0\bma_init_1987-06.pkl
Loading cached BMA predictions from: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\bma_cache2.0\bma_pred_1987-06.pkl
Processing 1987-07
Loading cached BMA initialization from: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\bma_cache2.0\bma_init_1987-07.pkl
Loading cached BMA predictions from: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\bma_cache2.0\bma_pred_1987-07.pkl
Processing 1987-08
Loading cached BMA initialization from: c:\Users\toros\OneDrive\Dokumenter\advanced_finance\speciale_repo\bma_cache2.0\bma_init_1987-08.pkl
Loading cached 

In [14]:
# --- Performance summary ------------------------------------------
summary_frames = []
for mname, rdict in results_by_method.items():
    s = po.summarize_backtest_results(rdict)
    # Replace monthly Sharpe with annualised version
    s["Sharpe Ratio"] = [annualised_sharpe(df["Portfolio Returns"]) for df in rdict.values()]
    s["Method"] = mname
    summary_frames.append(s.reset_index())

perf_df = pd.concat(summary_frames).set_index("Strategy_Key")
print("Strategy‑level performance (annualised SR):")
display(perf_df[["Method", "Sharpe Ratio", "Final Wealth"]].head(10))

Strategy‑level performance (annualised SR):


,Method,Sharpe Ratio,Final Wealth
Strategy_Key,,,
aggressive_2.5_0.2,PT BMA,1.195573,6.387481
conservative_2.5_0.2,PT BMA,1.202901,6.385689
aggressive_2.5_0.12,PT BMA,0.594445,6.405582
conservative_2.5_0.12,PT BMA,1.259536,6.422983
aggressive_1.99_0.2,PT BMA,1.403876,6.726685
aggressive_1.99_0.12,PT BMA,1.405223,7.220581
conservative_1.99_0.12,PT BMA,1.459246,7.210591
conservative_1.99_0.2,PT BMA,1.460986,5.121438
conservative_2.5_0.2,PT HistMean,1.224691,6.602531


In [15]:
# --- Detailed PT method tables ------------------------------------
pt_methods = ["PT BMA", "PT HistMean"]
for m in pt_methods:
    display(pd.DataFrame({
        "Sharpe Ratio": perf_df[perf_df.Method == m]["Sharpe Ratio"],
        "Final Wealth": perf_df[perf_df.Method == m]["Final Wealth"],
    }).sort_values("Sharpe Ratio", ascending=False).style.set_caption(f"{m} – Strategy‑level Results"))

,Sharpe Ratio,Final Wealth
Strategy_Key,,
conservative_1.99_0.2,1.460986,5.121438
conservative_1.99_0.12,1.459246,7.210591
aggressive_1.99_0.12,1.405223,7.220581
aggressive_1.99_0.2,1.403876,6.726685
conservative_2.5_0.12,1.259536,6.422983
conservative_2.5_0.2,1.202901,6.385689
aggressive_2.5_0.2,1.195573,6.387481
aggressive_2.5_0.12,0.594445,6.405582


,Sharpe Ratio,Final Wealth
Strategy_Key,,
aggressive_1.99_0.12,1.498473,6.745722
aggressive_1.99_0.2,1.483699,6.760718
conservative_1.99_0.12,1.464672,6.863662
conservative_1.99_0.2,1.434587,6.954635
aggressive_2.5_0.2,1.335694,6.399399
conservative_2.5_0.12,1.307921,6.511464
aggressive_2.5_0.12,1.249584,6.666820
conservative_2.5_0.2,1.224691,6.602531


In [16]:
# --- Average cumulative net‑return comparison ---------------------
cum_net_df = pd.DataFrame({m: mean_cumulative(r) for m, r in results_by_method.items()})
display(cum_net_df.tail())  # show last few rows

final_wealth = cum_net_df.iloc[-1].rename("Final Wealth")
display(final_wealth.to_frame().T.style.set_caption("Final Wealth (average of strategies)") )

,PT BMA,PT HistMean,MVP benchmark,MaxSharpe BMA,Naive 1/N
2016-08-01,6.382877,6.513266,5.033428,5.012590,5.857994
2016-09-01,6.393759,6.410099,5.026305,5.028105,5.849703
2016-10-01,6.278808,6.458614,5.060426,4.958563,5.889417
2016-11-01,6.341857,6.596531,5.236059,5.001876,6.093842
2016-12-01,6.485129,6.688119,5.269587,5.127538,6.132867


,PT BMA,PT HistMean,MVP benchmark,MaxSharpe BMA,Naive 1/N
Final Wealth,6.485129,6.688119,5.269587,5.127538,6.132867


In [17]:
# --- Average Certainty Equivalents -------------------------------
ce_df_all = ev.compare_certainty_equivalents(results_by_method, reference_series)
ce_summary = ev.summarize_certainty_equivalents(ce_df_all)
display(ce_summary.style.set_caption("Average Certainty Equivalent by Method"))

,CE Sum,CE Mean,CE Std Dev,CE Median
Method,,,,
Naive 1/N,0.000721,0.000721,nan,0.000721
MaxSharpe BMA,0.000593,0.000593,nan,0.000593
MVP benchmark,0.000426,0.000426,nan,0.000426
PT BMA,0.000137,0.000137,nan,0.000137
PT HistMean,0.000090,0.000090,nan,0.000090


In [23]:
# ── 1  Run the PT Factor back-test (replace with your own generator) ─────────
print("⏳ Back-testing PT by Factor model …")
#results_factor = po.resultgenerator_factor(          # ← use your actual func
#    lambda_values, gamma_values,
#    #factor_returns,                 # e.g. po.load_factor_returns(...)
#    historical_returns,
#    strategies, date_tag)

# ── 2  Collect every method in one dict ──────────────────────────────────────
results_by_method = {
    "PT BMA":            results_bma,
    "PT HistMean":       results_hist,
#    "PT Factor":         results_factor,           # ← newly added
    "MVP benchmark":     results_mvp,
    "MaxSharpe BMA":     results_bma_ms,
    "Naive 1/N":         results_naive,
}

# ── 3  Average cumulative net-return path per method ‐ and final wealth ─────
cum_net_df   = pd.DataFrame({m: mean_cumulative(r) for m, r in results_by_method.items()})
final_wealth = cum_net_df.iloc[-1].rename("Final Wealth")        # Series: index = method

# ── 4  Transposed table – each method in its own row ────────────────────────
final_wealth_tbl = final_wealth.to_frame(name="Final Wealth")    # already transposed
display(final_wealth_tbl.style.set_caption("Final Wealth (average of strategies) – transposed"))

# ── 5  OPTIONAL: add more benchmark metrics on the same rows ────────────────
# Annualised Sharpe (already in perf_df) – take mean across strategies
avg_sharpe = perf_df.groupby("Method")["Sharpe Ratio"].mean()

# Certainty Equivalent mean (already in ce_summary)
avg_ce = ce_summary["CE Mean"]

benchmark_df = pd.concat([final_wealth, avg_sharpe, avg_ce], axis=1)
benchmark_df.columns = ["Final Wealth", "Annualised Sharpe", "Average CE"]
benchmark_df = benchmark_df.loc[results_by_method.keys()]        # keep row order tidy
display(benchmark_df.style.set_caption("Benchmark metrics by method (transposed)"))

⏳ Back-testing PT by Factor model …


,Final Wealth
PT BMA,6.485129
PT HistMean,6.688119
MVP benchmark,5.269587
MaxSharpe BMA,5.127538
Naive 1/N,6.132867


,Final Wealth,Annualised Sharpe,Average CE
PT BMA,6.485129,1.247723,0.000137
PT HistMean,6.688119,1.374915,0.000090
MVP benchmark,5.269587,1.640097,0.000426
MaxSharpe BMA,5.127538,1.750960,0.000593
Naive 1/N,6.132867,1.673375,0.000721


In [20]:
# --- Plot cumulative net‑returns ----------------------------------
plt.figure(figsize=(10, 6))
for col in cum_net_df.columns:
    plt.plot(cum_net_df.index, cum_net_df[col], label=col)
plt.title("Cumulative Net Return – average of strategies per method")
plt.ylabel("Net wealth (Start = 1.0)")
plt.legend()
plt.show()

C:\Users\toros\AppData\Local\Temp\ipykernel_23828\3000813183.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
